## Обогащение и преобразования

### 1. Прочитайте файл JSON, который вы сохранили в ex02

- один из столбцов имеет тип float, поэтому давайте определим его формат в pandas с помощью pd.options.display.float_format: числа с плавающей точкой должны отображаться с двумя десятичными знаками
- в модели отсутствуют значения, ничего с ними не делайте

In [1]:
import pandas as pd
import numpy as np
df = pd.read_json('../data/auto.json')
pd.options.display.float_format = '{:.2f}'.format
print(df)

        CarNumber  Refund    Fines    Make    Model
0    Y163O8161RUS       2  3200.00    Ford    Focus
1     E432XX77RUS       1  6500.00  Toyota    Camry
2     7184TT36RUS       1  2100.00    Ford    Focus
3    X582HE161RUS       2  2000.00    Ford    Focus
4    E34877152RUS       2  6100.00    Ford    Focus
..            ...     ...      ...     ...      ...
926  Y163O8161RUS       2  1600.00    Ford    Focus
927  M0309X197RUS       1 22300.00    Ford    Focus
928  O673E8197RUS       2   600.00    Ford    Focus
929  8610T8154RUS       1  2000.00    Ford    Focus
930  H419XE197RUS       2  7860.99  Toyota  Corolla

[931 rows x 5 columns]


### 2. Oбогатить фрейм данных, используя выборку из этого фрейма данных

- создать выборку из 200 новых наблюдений с random_state = 21
 в выборке не должно быть новых комбинаций номера автомобиля, марки и модели, поэтому весь набор данных будет согласованным в этих терминах
 нет ограничений по возврату и штрафам, вы можете взять любое значение из этих столбцов наугад и использовать его для любого номера автомобиля


In [2]:
sampled_df = df.sample(n=200, random_state=21)
unique_combinations = df[['CarNumber', 'Make', 'Model']].drop_duplicates()
sampled_df = sampled_df.merge(unique_combinations, on=['CarNumber', 'Make', 'Model'], how='inner')
sampled_df['Refund'] = np.random.choice(df['Refund'], size=len(sampled_df))
sampled_df['Fines'] = np.random.choice(df['Fines'], size=len(sampled_df))
print(sampled_df)

        CarNumber  Refund    Fines   Make    Model
0    T6488M116RUS       2  6600.00   Ford    Focus
1    98087H178RUS       1  8600.00   Ford    Focus
2    O134HE197RUS       1   100.00   Ford    Focus
3    H889YK197RUS       2 17500.00   Ford    Focus
4    Y195O8161RUS       1  6000.00   Ford    Focus
..            ...     ...      ...    ...      ...
195  96659X178RUS       1  1000.00   Ford    Focus
196  Y650E8750RUS       1 12600.00  Skoda  Octavia
197  9428EX178RUS       1  6800.00   Ford    Focus
198  E396MO152RUS       1  7860.99   Ford    Focus
199  E394MO152RUS       2   600.00   Ford    Focus

[200 rows x 5 columns]


- объединить выборку с исходным фреймом данных в новый фрейм данных con-cat_rows

In [3]:
concat_rows = pd.concat([df, sampled_df])
print(concat_rows)

        CarNumber  Refund    Fines    Make    Model
0    Y163O8161RUS       2  3200.00    Ford    Focus
1     E432XX77RUS       1  6500.00  Toyota    Camry
2     7184TT36RUS       1  2100.00    Ford    Focus
3    X582HE161RUS       2  2000.00    Ford    Focus
4    E34877152RUS       2  6100.00    Ford    Focus
..            ...     ...      ...     ...      ...
195  96659X178RUS       1  1000.00    Ford    Focus
196  Y650E8750RUS       1 12600.00   Skoda  Octavia
197  9428EX178RUS       1  6800.00    Ford    Focus
198  E396MO152RUS       1  7860.99    Ford    Focus
199  E394MO152RUS       2   600.00    Ford    Focus

[1131 rows x 5 columns]


### 3. Oбогатить фрейм данных concat_rows новым столбцом с полученными данными

- создайте ряд с названием Год, используя случайные целые числа от 1980 до 2019
- используйте np.random.seed(21) перед генерацией лет
- соедините ряд с фреймом данных и назовите его штрафы

In [4]:
np.random.seed(21)
Year = pd.Series(np.random.randint(1980, 2019, size = 925), name = 'Year')
concat_rows.reset_index(drop=True, inplace=True)
fines = pd.concat([concat_rows, Year], axis=1)
print(fines)

         CarNumber  Refund    Fines    Make    Model    Year
0     Y163O8161RUS       2  3200.00    Ford    Focus 1989.00
1      E432XX77RUS       1  6500.00  Toyota    Camry 1995.00
2      7184TT36RUS       1  2100.00    Ford    Focus 1984.00
3     X582HE161RUS       2  2000.00    Ford    Focus 2015.00
4     E34877152RUS       2  6100.00    Ford    Focus 2014.00
...            ...     ...      ...     ...      ...     ...
1126  96659X178RUS       1  1000.00    Ford    Focus     NaN
1127  Y650E8750RUS       1 12600.00   Skoda  Octavia     NaN
1128  9428EX178RUS       1  6800.00    Ford    Focus     NaN
1129  E396MO152RUS       1  7860.99    Ford    Focus     NaN
1130  E394MO152RUS       2   600.00    Ford    Focus     NaN

[1131 rows x 6 columns]


### 4. Oбогатить фрейм данных данными из другого фрейма данных

**4.1 Cоздать новый фрейм данных с номерами автомобилей и их владельцами**
- получить самые популярные фамилии ( файл named.json можно найти во вложениях ) в США
- создайте новую серию с фамилиями (они не должны содержать специальных символов, таких как запятые, скобки и т. д.) из собранных вами данных, количество должно быть равно количеству уникальных номеров автомобилей, использующих выборку (используйте random_state = 21)
- создайте владельцев фрейма данных с 2 столбцами: CarNumber и SURNAME

In [5]:
import pandas as pd
surnames = pd.read_json('../../datasets/surname.json')
surnames.columns = ['NAME', 'COUNT', 'RANK']
surnames = surnames.drop(0).reset_index(drop=True)
surnames['RANK'] = surnames['RANK'].astype(int)
surnames = surnames.sort_values(by='RANK')
print(f"Самые популярные фамилии: \n\n{surnames}")

Самые популярные фамилии: 

        NAME    COUNT  RANK
84     SMITH  2442977     1
40   JOHNSON  1932812     2
95  WILLIAMS  1625252     3
8      BROWN  1437026     4
41     JONES  1425470     5
..       ...      ...   ...
59     MYERS   229895    96
47      LONG   229374    97
79      ROSS   229368    98
24    FOSTER   227764    99
39   JIMENEZ   227118   100

[100 rows x 3 columns]


In [6]:
np.random.seed(21)
''' Из 03 '''
Year = pd.Series(np.random.randint(1980, 2019, size = 925), name = 'Year')
concat_rows.reset_index(drop=True, inplace=True)
fines = pd.concat([concat_rows, Year], axis=1)
''' Иначе fines потерялась '''
unique_car_count = fines['CarNumber'].nunique()
sampled_surnames = surnames['NAME'].sample(n=unique_car_count, replace=True, random_state=21)
surname_series = pd.Series(sampled_surnames.values, name = 'Surname')
print(surname_series)

0           KELLY
1      RICHARDSON
2            CRUZ
3           JONES
4          CARTER
          ...    
526      MARTINEZ
527         YOUNG
528        GARCIA
529           LEE
530          CRUZ
Name: Surname, Length: 531, dtype: object


In [7]:
owners = pd.DataFrame({
    'CarNumber': fines['CarNumber'].unique(),
    'Surname': sampled_surnames.values
})
print(owners)

        CarNumber     Surname
0    Y163O8161RUS       KELLY
1     E432XX77RUS  RICHARDSON
2     7184TT36RUS        CRUZ
3    X582HE161RUS       JONES
4    E34877152RUS      CARTER
..            ...         ...
526  7367C8197RUS    MARTINEZ
527  9398MM199RUS       YOUNG
528  O136HO197RUS      GARCIA
529  M0309X197RUS         LEE
530  O673E8197RUS        CRUZ

[531 rows x 2 columns]


**4.2 Добавьте еще 5 наблюдений к фрейму данных о штрафах (предложите свои собственные идеи CarNumber и т. д.)**

In [8]:
additional_fines = pd.DataFrame({
    'CarNumber': ['1111C8197RUS','2222C8197RUS', '33339Y50RUS', '4444MM116RUS', '5555M7161RUS'],
    'Refund': [2, 1, 2, 1, 2],
    'Fines': [500, 100, 10100, 2400, 7800],
    'Make': ['Toyota', 'Ford', 'Volkswagen', 'Skoda', 'Porshe'],
    'Model': ['Corolla', 'Focus', 'Golf', 'Octavia', '911'],
    'Year': [2007, 1997, 1984, 2017, 2077]
})
fines = pd.concat([fines, additional_fines], ignore_index=True)
print(fines)

         CarNumber  Refund    Fines        Make    Model    Year
0     Y163O8161RUS       2  3200.00        Ford    Focus 1989.00
1      E432XX77RUS       1  6500.00      Toyota    Camry 1995.00
2      7184TT36RUS       1  2100.00        Ford    Focus 1984.00
3     X582HE161RUS       2  2000.00        Ford    Focus 2015.00
4     E34877152RUS       2  6100.00        Ford    Focus 2014.00
...            ...     ...      ...         ...      ...     ...
1131  1111C8197RUS       2   500.00      Toyota  Corolla 2007.00
1132  2222C8197RUS       1   100.00        Ford    Focus 1997.00
1133   33339Y50RUS       2 10100.00  Volkswagen     Golf 1984.00
1134  4444MM116RUS       1  2400.00       Skoda  Octavia 2017.00
1135  5555M7161RUS       2  7800.00      Porshe      911 2077.00

[1136 rows x 6 columns]


**4.3 Удалите из датафрейма последние 20 наблюдений от владельцев и добавьте 3 новых наблюдения (они не совпадают с теми, которые вы добавляете в датафрейм штрафов)**

In [9]:
owners = owners.iloc[:-20]
add_owners = pd.DataFrame({
    'CarNumber': ['666660197RUS', '777770197RUS', '888880197RUS'],
    'Surname': ['MISSI', 'SIPPI', 'SILVERHAND']
})
owners = pd.concat([owners, add_owners], ignore_index=True)
print(owners)

        CarNumber     Surname
0    Y163O8161RUS       KELLY
1     E432XX77RUS  RICHARDSON
2     7184TT36RUS        CRUZ
3    X582HE161RUS       JONES
4    E34877152RUS      CARTER
..            ...         ...
509  7360C8197RUS      FOSTER
510  E29677161RUS      WRIGHT
511  666660197RUS       MISSI
512  777770197RUS       SIPPI
513  888880197RUS  SILVERHAND

[514 rows x 2 columns]


**4.4 Объедините оба фрейма данных:**
- новый фрейм данных должен содержать только те номера автомобилей, которые существуют в обоих фреймах данных
- новый фрейм данных должен содержать все номера автомобилей, которые существуют в обоих фреймах данных
- новый фрейм данных должен содержать только номера автомобилей из фрейма данных о штрафах
- новый фрейм данных должен содержать только номера автомобилей из фрейма данных владельцев

In [10]:
both_carnum = pd.merge(fines, owners, how='inner', on='CarNumber')
all_carnum = pd.merge(fines, owners, how='outer', on='CarNumber')
fines_carnum = pd.merge(fines, owners, how='left', on='CarNumber')
owners_carnum = pd.merge(fines, owners, how='right', on='CarNumber')
print(f"Только номера автомобилей, которые существуют в обоих фреймах данных: \n{both_carnum}")
print(f"\nВсе номера автомобилей, которые существуют в обоих фреймах данных: \n{all_carnum}")
print(f"\nТолько номера автомобилей из фрейма данных о штрафах: \n{fines_carnum}")
print(f"\nТолько номера автомобилей из фрейма данных владельцев: \n{owners_carnum}")

Только номера автомобилей, которые существуют в обоих фреймах данных: 
         CarNumber  Refund    Fines    Make    Model    Year     Surname
0     Y163O8161RUS       2  3200.00    Ford    Focus 1989.00       KELLY
1     Y163O8161RUS       2  1600.00    Ford    Focus     NaN       KELLY
2      E432XX77RUS       1  6500.00  Toyota    Camry 1995.00  RICHARDSON
3      E432XX77RUS       2 13000.00  Toyota    Camry 1993.00  RICHARDSON
4      E432XX77RUS       2 37800.00  Toyota    Camry     NaN  RICHARDSON
...            ...     ...      ...     ...      ...     ...         ...
1101  H259HY161RUS       1   900.00  Toyota  Corolla 2008.00        KING
1102  K889HY161RUS       2 46200.00    Ford   Mondeo 2014.00      GARCIA
1103  O421OT161RUS       1  1200.00   Skoda  Octavia 2014.00     ROBERTS
1104  7360C8197RUS       2  9600.00    Ford    Focus 2014.00      FOSTER
1105  E29677161RUS       2  6000.00    Ford    Focus 1995.00      WRIGHT

[1106 rows x 7 columns]

Все номера автомобилей, кот

### 5. Cоздайте сводную таблицу из фрейма данных штрафов со всеми годами.
- сохранить данные о штрафах и владельцах в CSV-файлы без индекса

In [11]:
pivot_table = fines.pivot_table(
    values='Fines',
    index=['Make', 'Model'],
    columns='Year',
    aggfunc='sum'
)
print(pivot_table)

Year                 1980.00  1981.00   1982.00   1983.00   1984.00   1985.00  \
Make       Model                                                                
Audi                     NaN      NaN       NaN       NaN       NaN       NaN   
BMW                      NaN      NaN       NaN       NaN       NaN       NaN   
Ford       Focus   110421.98 97160.99 104021.98 330260.99 100782.97 102321.98   
           Mondeo        NaN  8600.00       NaN       NaN       NaN  34400.00   
Porshe     911           NaN      NaN       NaN       NaN       NaN       NaN   
Skoda      Octavia   8500.00      NaN  13460.99  19800.00   3000.00 152000.00   
Toyota     Camry         NaN  7860.99       NaN       NaN       NaN   1000.00   
           Corolla       NaN      NaN   3200.00       NaN  34300.00       NaN   
Volkswagen               NaN      NaN       NaN       NaN       NaN       NaN   
           Golf      4600.00      NaN    300.00    200.00  10100.00   4000.00   
           Jetta         NaN

In [12]:
fines.to_csv('../data/fines.csv', index=False)
owners.to_csv('../data/owners.csv', index=False)
print('Данные о штрафах и владельцах сохранены в CSV-файлы')

Данные о штрафах и владельцах сохранены в CSV-файлы
